## Packages
Install necessary packages: numpy, control, and multiprocessing.

In [1]:
!pip install -r requirements.txt -U


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import necessary packages
import re

import numpy as np
import control as ct

from typing import Callable, Tuple

import matplotlib.pyplot as plt

## Plot
Given a transfer function **H**, frequency range **\[start, end\]**, and propagation delay **delay**, we numerically approximate the transfer function given a square wave amplifier, which theoretically produces an infinite number of harmonics. If there is no propagation delay or it is already approximated in the transfer function, **delay** defaults to the floating point ε to approximate the limit form mentioned in Putzey's paper.

In [3]:
from modules.calc_resp import precompute_consts, calc_resp

def calc(
    H: ct.TransferFunction, 
    start: float, 
    end: float, 
    delay=np.finfo(np.longdouble).eps,
    num_freqs=1_000,
    num_duty_cycles=100,
    num_harmonics=100.
) -> Tuple[np.ndarray]:
    consts = list(precompute_consts(num_freqs, num_duty_cycles, num_harmonics, (start, end)))
    
    mags, phs, osc_fs, dcins, dcgains = calc_resp(np.array(H.num[0])[:,::-1], np.array(H.den[0])[:,::-1], np.array([delay]), *consts)
    
    consts[0] = np.imag(consts[0]) / (2 * np.pi)
    osc_fs /= 2 * np.pi
    phs -= np.pi

    return (mags[0], phs[0], osc_fs[0], dcins[0], dcgains[0]), consts[:3]

In [4]:
def plot(
    H: ct.TransferFunction, 
    start: float, 
    end: float, 
    delay=np.finfo(np.longdouble).eps
) -> None:
    """ Takes a SISO transfer function `H` and plots phase response and loop gain vs.
    frequency, oscillation frequency vs. duty cycle, DC transfer curve, and loop gain 
    vs. duty cycle. """

    (mags, phs, osc_fs, dcins, dcgains), (omega, hs, ns) = calc(H, start, end, delay)
    
    # Plotting
    fig, ((ax_ph, ax_dcgain), (ax_osc, ax_dcin)) = plt.subplots(2,2)

    # Set figure labels
    ax_osc.set_ylabel('Osc. Freq. (Hz)')
    ax_osc.set_xlabel('Duty Cycle')

    ax_ph.set_xlabel('Frequency (Hz)')
    ax_ph.set_ylabel('Phase (°)')
    ax_mag = ax_ph.twinx()
    ax_mag.set_ylabel('Magnitude (dB)')

    # Plot phase
    cs = ['b','g','c','y']
    for i, p in enumerate(phs[-1:0:-(hs.size // 5),:]):
        ax_ph.plot(omega, np.degrees(np.unwrap(p)), f'{cs[i % len(cs)]}--')

    ax_ph.plot([start, end], [-180, -180], 'k:')
    ax_ph.set_xscale('log')

    max_ph = max(np.ceil(np.degrees(np.max(phs))), 0) + 10
    ax_ph.set_yticks([0, -180, -360])
    ax_ph.set_ylim(-360-max_ph, max_ph)

    # Plot oscillation frequencies
    ax_osc.plot(
        np.concatenate((hs, 1-hs[-1::-1])),
        np.concatenate((osc_fs, osc_fs[-1::-1]))
    )

    # Plot dcin
    ax_dcin.plot(
        np.concatenate((-dcins, dcins[-1::-1])),
        np.concatenate((2 * hs - 1, - 2 * hs[-1::-1] + 1))
    )

    # Plot dcgain
    ax_dcgain.plot(
        np.concatenate((2 * hs - 1, - 2 * hs[-1::-1] + 1)),
        np.concatenate((dcgains, dcgains[-1::-1]))
    )

    # Plot magnitudes
    mag_scaled = 20 * np.log10(dcgains[-1] * mags[-1])
    ax_mag.plot(omega, mag_scaled, 'r-')
    
    max_mag = round(max(
        *map(
            np.abs, 
            (np.min(mag_scaled), np.max(mag_scaled))
        ), 
        0
    )) + 3
    ax_mag.set_ylim(-max_mag, max_mag)

    fig.tight_layout()



## Transfer Functions

As per Bruno Putzey's 2011 paper titled ["Global Modulated Self-Oscillating Amplifier with Improved Linearity"](https://www.hypex.nl/media/3f/62/4a/1682342035/Globally%20modulated%20self-oscillating%20amplifier.pdf), we treat our Class-D amplifier as a square wave oscillator wrapped with a linear function. For simplicity's sake, we further split said linear function into three serial sections–the **propagation delay**, **low pass filter**, and **feedback network**. The **propagation delay** is calculated during numerical evaluation in order to avoid using a Padé approximation. Please define functions constructing transfer functions based on component values for the **low pass filter** and **feedback network** below.

## Workspace

In [5]:
from modules.sapwin import Sapwin

problem = Sapwin(filename='../sym_analysis/SapWin/class_d_2.out', n_fs=1_000, n_ns=200)

In [10]:
from pymoo.algorithms.soo.nonconvex.de import DE

algorithm = DE(
    pop_size=1_000,
    CR=0.9,
    dither='vector',
    jitter=True
)

In [ ]:
from pymoo.optimize import minimize

res = minimize(
    problem,
    algorithm,
    termination=('n_gen', 100),
    seed=1,
    save_history=True,
    verbose=True
)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |     1000 |           INF |  0.0831944000
     2 |     2000 |           INF |  0.0072093922
     3 |     3000 |           INF |  0.0038254089
     4 |     4000 |           INF |  0.0036959973
     5 |     5000 |  0.0746866911 |  0.0034058146
     6 |     6000 |  0.0279703229 |  0.0033449517
     7 |     7000 |  0.0083276017 |  0.0033405563
     8 |     8000 |  0.0051327960 |  0.0033297674
     9 |     9000 |  0.0042231689 |  0.0033279090
    10 |    10000 |  0.0037719064 |  0.0033160877
    11 |    11000 |  0.0035883421 |  0.0033144409
    12 |    12000 |  0.0034781342 |  0.0033129027
    13 |    13000 |  0.0034111635 |  0.0033123340
    14 |    14000 |  0.0033690134 |  0.0033103129
    15 |    15000 |  0.0033454921 |  0.0033100238
    16 |    16000 |  0.0033294915 |  0.0033089295
    17 |    17000 |  0.0033207577 |  0.0033034064
    18 |    18000 |  0.0033125946 |  0.0032979043
    19 |    19000 |  0.0033042025 |  0.0032886751


In [ ]:
X = res.X
F = res.F

In [ ]:
vals = X[:-1][None,:]
delay = X[-1]

num = problem._calc_num(vals)[0,::-1]
den = problem._calc_den(vals)[0,::-1]

plot(ct.tf(num, den), 10, 2e7, delay)